In [1]:
from os import listdir
import numpy as np
import tensorflow as tf
from tensorflow import keras
from numpy import asarray
from numpy import vstack
from tensorflow.keras.preprocessing.image import img_to_array
from keras.preprocessing.image import load_img
from numpy import savez_compressed


C:\Users\Aryan\Anaconda3\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\Aryan\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.NOIJJG62EMASZI6NYURL6JBKM4EVBGM7.gfortran-win_amd64.dll
C:\Users\Aryan\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.QVLO2T66WEPI7JZ63PS3HMOHFEY472BC.gfortran-win_amd64.dll
  stacklevel=1)


ModuleNotFoundError: No module named 'keras'

In [ ]:
class CGAN:
    @staticmethod
    def discriminator(image_shape):
        init = RandomNormal(stddev = 0.02)  #initializer
        in_image = Input(shape=image_shape)
        
        d = Conv2D(64,(4,4),strides=(2,2),padding='same', kernel_initializer=init)
        d = LeakyReLU(alpha=0.2)(d)
        
        d = Conv2D(128, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(d)
        d = InstanceNormalization(axis=-1)(d)
        d = LeakyReLU(alpha=0.2)(d)
        
        d = Conv2D(256, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(d)
        d = InstanceNormalization(axis=-1)(d)
        d = LeakyReLU(alpha=0.2)(d)
        
        d = Conv2D(512, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(d)
        d = InstanceNormalization(axis=-1)(d)
        d = LeakyReLU(alpha=0.2)(d)
        
        d = Conv2D(512, (4,4), padding='same', kernel_initializer=init)(d)
        d = InstanceNormalization(axis=-1)(d)
        d = LeakyReLU(alpha=0.2)(d)
        
        patch_out = Conv2D(1, (4,4), padding='same', kernel_initializer=init)(d)
        
        model = Model(in_image, patch_out)
        
        model.compile(loss='mse', optimizer=Adam(lr=0.0002, beta_1=0.5), loss_weights=[0.5])
        return model